In [4]:
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader

import torchvision
import os
from matplotlib import pyplot as plt
import numpy as np

print(torch.__version__)
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

2.0.0+cu118


In [5]:
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [6]:
# Download training data from open datasets.
training_data = torchvision.datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=torchvision.transforms.ToTensor(),
)


# Download test data from open datasets.
test_data = torchvision.datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=torchvision.transforms.ToTensor(),
)


100%|██████████| 26421880/26421880 [00:00<00:00, 115762124.44it/s]


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 4620763.78it/s]

Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw




100%|██████████| 4422102/4422102 [00:00<00:00, 60937806.31it/s]


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 12459478.93it/s]


Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



In [7]:
# from: https://www.kaggle.com/code/pankajj/fashion-mnist-with-pytorch-93-accuracy
def output_label(label):
    output_mapping = {
                 0: "T-shirt/Top",
                 1: "Trouser",
                 2: "Pullover",
                 3: "Dress",
                 4: "Coat",
                 5: "Sandal",
                 6: "Shirt",
                 7: "Sneaker",
                 8: "Bag",
                 9: "Ankle Boot"
                 }
    input = (label.item() if type(label) == torch.Tensor else label)
    return output_mapping[input]

In [8]:
# Feed Forward Neural Network

# model architecture from: https://www.kaggle.com/code/pankajj/fashion-mnist-with-pytorch-93-accuracy

class Classifier(nn.Module):
    def __init__(self, input_shape : torch.Size, hidden_sizes : list[int], num_classes : int, activation_function, loss_fn):
        super().__init__()
        self.conv_layer1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            activation_function,
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.conv_layer2 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3),
            nn.BatchNorm2d(64),
            activation_function,
            nn.MaxPool2d(2)
        )
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential( # todo use hidden sizes
            nn.Linear(64*6*6, 600),
            nn.Dropout(0.25),
            activation_function,
            nn.Linear(600, 120),
            activation_function,
            nn.Linear(120, num_classes) # no softmax necessary
        )

        self.num_classes = num_classes
        self.loss_fn = loss_fn

    def forward(self, x : torch.Tensor) -> torch.Tensor:
        x = self.conv_layer1(x)
        x = self.conv_layer2(x)
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits


    def train_network(self, dataloader : DataLoader, optimizer : torch.optim.Optimizer):
        size = len(dataloader.dataset)
        self.train()
        for batch, (X, y) in enumerate(dataloader):
            X, y = X.to(device), y.to(device)

            # Compute prediction error
            y_pred = self.forward(X)
            loss = self.loss_fn(y_pred, F.one_hot(y, self.num_classes).type(y_pred.dtype))

            # Backpropagation
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            if batch % 100 == 0:
                loss, current = loss.item(), (batch + 1) * len(X)
                print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

    def test(self, dataloader : DataLoader):
        num_batches = len(dataloader)
        self.eval()
        test_loss = 0
        num_right = 0
        num_correct = 0
        with torch.no_grad():
            for X, y in dataloader:
                X, y = X.to(device), y.to(device)
                y_pred  = self.forward(X)
                test_loss += self.loss_fn(y_pred , F.one_hot(y, self.num_classes).type(y_pred.dtype)).item()
                num_correct += (torch.argmax(y_pred, dim=1) == y).sum().item()
        test_loss /= num_batches
        accuracy = num_correct / len(dataloader.dataset)
        print(f"Test Error: Avg loss: {test_loss:>8f} Accuracy: {float(accuracy):>8f} \n")

In [9]:
batch_size = 2

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break


Shape of X [N, C, H, W]: torch.Size([2, 1, 28, 28])
Shape of y: torch.Size([2]) torch.int64


In [11]:
size = torch.Size((1,28,28))
num_classes = 10
epochs = 5


classifier = Classifier(size, [], num_classes, nn.ReLU(), nn.CrossEntropyLoss()).to(device)
optimizer = torch.optim.Adam(classifier.parameters(), lr=1e-3)

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    classifier.train_network(train_dataloader, optimizer)
    classifier.test(test_dataloader)
print("Done!")


Epoch 1
-------------------------------
loss: 2.236966  [    2/60000]
loss: 3.268069  [  202/60000]
loss: 1.389572  [  402/60000]
loss: 0.654887  [  602/60000]
loss: 1.613296  [  802/60000]
loss: 0.460688  [ 1002/60000]
loss: 0.743810  [ 1202/60000]
loss: 2.145666  [ 1402/60000]
loss: 3.367494  [ 1602/60000]
loss: 0.362263  [ 1802/60000]
loss: 0.277623  [ 2002/60000]
loss: 0.056401  [ 2202/60000]
loss: 2.257266  [ 2402/60000]
loss: 1.535370  [ 2602/60000]
loss: 0.878837  [ 2802/60000]
loss: 0.625087  [ 3002/60000]
loss: 0.574371  [ 3202/60000]
loss: 0.631517  [ 3402/60000]
loss: 1.644186  [ 3602/60000]
loss: 0.317648  [ 3802/60000]
loss: 0.149533  [ 4002/60000]
loss: 0.005241  [ 4202/60000]
loss: 0.983753  [ 4402/60000]
loss: 0.000120  [ 4602/60000]
loss: 0.870985  [ 4802/60000]
loss: 0.336335  [ 5002/60000]
loss: 0.104162  [ 5202/60000]
loss: 0.297630  [ 5402/60000]
loss: 0.655000  [ 5602/60000]
loss: 0.034855  [ 5802/60000]
loss: 0.776370  [ 6002/60000]
loss: 0.000382  [ 6202/60000]


In [13]:
batch_size = 16

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)


size = torch.Size((1,28,28))


classifier = Classifier(size, [], num_classes, nn.ReLU(), nn.CrossEntropyLoss()).to(device)
optimizer = torch.optim.Adam(classifier.parameters(), lr=1e-3)

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    classifier.train_network(train_dataloader, optimizer)
    classifier.test(test_dataloader)
print("Done!")

Epoch 1
-------------------------------
loss: 2.351409  [   16/60000]
loss: 0.743569  [ 1616/60000]
loss: 0.634741  [ 3216/60000]
loss: 0.477763  [ 4816/60000]
loss: 0.388747  [ 6416/60000]
loss: 0.735409  [ 8016/60000]
loss: 0.495244  [ 9616/60000]
loss: 0.162244  [11216/60000]
loss: 0.587680  [12816/60000]
loss: 0.231612  [14416/60000]
loss: 0.804324  [16016/60000]
loss: 0.335152  [17616/60000]
loss: 0.155147  [19216/60000]
loss: 0.581478  [20816/60000]
loss: 0.536934  [22416/60000]
loss: 0.588689  [24016/60000]
loss: 0.388161  [25616/60000]
loss: 0.483898  [27216/60000]
loss: 0.296262  [28816/60000]
loss: 0.250107  [30416/60000]
loss: 0.584727  [32016/60000]
loss: 0.138394  [33616/60000]
loss: 0.442564  [35216/60000]
loss: 0.276764  [36816/60000]
loss: 0.050868  [38416/60000]
loss: 0.269830  [40016/60000]
loss: 0.390101  [41616/60000]
loss: 0.420953  [43216/60000]
loss: 0.320305  [44816/60000]
loss: 0.250185  [46416/60000]
loss: 0.089710  [48016/60000]
loss: 0.413710  [49616/60000]


In [14]:
batch_size = 32

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)


size = torch.Size((1,28,28))


classifier = Classifier(size, [], num_classes, nn.ReLU(), nn.CrossEntropyLoss()).to(device)
optimizer = torch.optim.Adam(classifier.parameters(), lr=1e-3)

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    classifier.train_network(train_dataloader, optimizer)
    classifier.test(test_dataloader)
print("Done!")

Epoch 1
-------------------------------
loss: 2.330386  [   32/60000]
loss: 0.646023  [ 3232/60000]
loss: 0.403816  [ 6432/60000]
loss: 0.573532  [ 9632/60000]
loss: 0.375540  [12832/60000]
loss: 0.621366  [16032/60000]
loss: 0.382900  [19232/60000]
loss: 0.260836  [22432/60000]
loss: 0.441003  [25632/60000]
loss: 0.409191  [28832/60000]
loss: 0.490514  [32032/60000]
loss: 0.410852  [35232/60000]
loss: 0.329305  [38432/60000]
loss: 0.355550  [41632/60000]
loss: 0.513526  [44832/60000]
loss: 0.291051  [48032/60000]
loss: 0.258055  [51232/60000]
loss: 0.477261  [54432/60000]
loss: 0.281667  [57632/60000]
Test Error: Avg loss: 0.360336 Accuracy: 0.869200 

Epoch 2
-------------------------------
loss: 0.178816  [   32/60000]
loss: 0.341407  [ 3232/60000]
loss: 0.174519  [ 6432/60000]
loss: 0.291109  [ 9632/60000]
loss: 0.169045  [12832/60000]
loss: 0.432301  [16032/60000]
loss: 0.277943  [19232/60000]
loss: 0.248064  [22432/60000]
loss: 0.306333  [25632/60000]
loss: 0.283087  [28832/60000

In [15]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break


size = torch.Size((1,28,28))


classifier = Classifier(size, [], num_classes, nn.ReLU(), nn.CrossEntropyLoss()).to(device)
optimizer = torch.optim.Adam(classifier.parameters(), lr=1e-3)

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    classifier.train_network(train_dataloader, optimizer)
    classifier.test(test_dataloader)
print("Done!")

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64
Epoch 1
-------------------------------
loss: 2.295302  [   64/60000]
loss: 0.430262  [ 6464/60000]
loss: 0.289053  [12864/60000]
loss: 0.465057  [19264/60000]
loss: 0.457087  [25664/60000]
loss: 0.351757  [32064/60000]
loss: 0.343226  [38464/60000]
loss: 0.515788  [44864/60000]
loss: 0.404897  [51264/60000]
loss: 0.299663  [57664/60000]
Test Error: Avg loss: 0.320517 Accuracy: 0.883300 

Epoch 2
-------------------------------
loss: 0.143424  [   64/60000]
loss: 0.402124  [ 6464/60000]
loss: 0.172932  [12864/60000]
loss: 0.314678  [19264/60000]
loss: 0.486271  [25664/60000]
loss: 0.356475  [32064/60000]
loss: 0.265656  [38464/60000]
loss: 0.329936  [44864/60000]
loss: 0.291834  [51264/60000]
loss: 0.213167  [57664/60000]
Test Error: Avg loss: 0.365378 Accuracy: 0.865300 

Epoch 3
-------------------------------
loss: 0.209710  [   64/60000]
loss: 0.310169  [ 6464/60000]
loss: 0.109971  [12864

In [16]:
batch_size = 256

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

# Train Classifier (SGD)
size = torch.Size((1,28,28))
num_classes = 10


classifier = Classifier(size, [], num_classes, nn.ReLU(), nn.CrossEntropyLoss()).to(device)
optimizer = torch.optim.Adam(classifier.parameters(), lr=1e-3)

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    classifier.train_network(train_dataloader, optimizer)
    classifier.test(test_dataloader)
print("Done!")

Shape of X [N, C, H, W]: torch.Size([256, 1, 28, 28])
Shape of y: torch.Size([256]) torch.int64
Epoch 1
-------------------------------
loss: 2.320994  [  256/60000]
loss: 0.342065  [25856/60000]
loss: 0.329342  [51456/60000]
Test Error: Avg loss: 0.360747 Accuracy: 0.869200 

Epoch 2
-------------------------------
loss: 0.275018  [  256/60000]
loss: 0.266455  [25856/60000]
loss: 0.288956  [51456/60000]
Test Error: Avg loss: 0.341543 Accuracy: 0.875200 

Epoch 3
-------------------------------
loss: 0.236373  [  256/60000]
loss: 0.221095  [25856/60000]
loss: 0.261111  [51456/60000]
Test Error: Avg loss: 0.321105 Accuracy: 0.886100 

Epoch 4
-------------------------------
loss: 0.217084  [  256/60000]
loss: 0.186538  [25856/60000]
loss: 0.251768  [51456/60000]
Test Error: Avg loss: 0.290621 Accuracy: 0.895800 

Epoch 5
-------------------------------
loss: 0.172459  [  256/60000]
loss: 0.189868  [25856/60000]
loss: 0.207183  [51456/60000]
Test Error: Avg loss: 0.281822 Accuracy: 0.898

In [17]:
batch_size = 1024

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

# Train Classifier (SGD)
size = torch.Size((1,28,28))
num_classes = 10



classifier = Classifier(size, [], num_classes, nn.ReLU(), nn.CrossEntropyLoss()).to(device)
optimizer = torch.optim.Adam(classifier.parameters(), lr=1e-3)

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    classifier.train_network(train_dataloader, optimizer)
    classifier.test(test_dataloader)
print("Done!")

Shape of X [N, C, H, W]: torch.Size([1024, 1, 28, 28])
Shape of y: torch.Size([1024]) torch.int64
Epoch 1
-------------------------------
loss: 2.309053  [ 1024/60000]
Test Error: Avg loss: 0.375022 Accuracy: 0.865000 

Epoch 2
-------------------------------
loss: 0.334994  [ 1024/60000]
Test Error: Avg loss: 0.312701 Accuracy: 0.890000 

Epoch 3
-------------------------------
loss: 0.258511  [ 1024/60000]
Test Error: Avg loss: 0.293876 Accuracy: 0.894000 

Epoch 4
-------------------------------
loss: 0.236592  [ 1024/60000]
Test Error: Avg loss: 0.264101 Accuracy: 0.905300 

Epoch 5
-------------------------------
loss: 0.207143  [ 1024/60000]
Test Error: Avg loss: 0.260746 Accuracy: 0.907300 

Done!
